In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
import joblib
import os

file_path = 'd:/data/weather/weather_clothing.csv'
os.makedirs('d:/data/weather', exist_ok=True)

df = pd.read_csv(file_path)

le_rain = LabelEncoder()
df['rain_encoded'] = le_rain.fit_transform(df['rain'])

le_cloth = LabelEncoder()
df['clothing_encoded'] = le_cloth.fit_transform(df['clothing'])

X = df[['temperature', 'humidity', 'wind_speed', 'rain_encoded']]
y = df['clothing_encoded']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_temp, y_train, y_temp = train_test_split(X_scaled, y, test_size=0.4, random_state=42, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

y_train_cat = tf.keras.utils.to_categorical(y_train)
y_val_cat = tf.keras.utils.to_categorical(y_val)
y_test_cat = tf.keras.utils.to_categorical(y_test)

model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(X.shape[1],)))
model.add(Dropout(0.3))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(len(np.unique(y)), activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

epochs = 50
batch_size = 32

history = model.fit(X_train, y_train_cat,
validation_data=(X_val, y_val_cat), 
epochs=epochs, 
batch_size = batch_size, verbose = 1)

plt.plot(history.history['accuracy'], label='train acc')
plt.plot(history.history['val_accuracy'], label='val acc')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training History')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig('d:/data/weather/training_history.png')
plt.show()

test_loss, test_acc = model.evaluate(X_test, y_test_cat, verbose=0)
print(f"Test Accuracy: {test_acc: .4f}")

model.save('d:/data/weather/weather_clothing_model.keras')
joblib.dump(scaler, 'd:/data/weather/weather_scaler.pkl')
joblib.dump(le_cloth, 'd:/data/weather/clothing_encoder.pkl')
joblib.dump(le_rain, 'd:/data/weather/rain_encoder.pkl')